In [1]:
#!pip install razdel # сегоментация русскоязычного текста

сегментация русскоязычного текста

In [6]:
#!pip install pymorphy2 

морфологический анализатор для русского языка, написанный на языке Python и использующий словари из OpenCorpora

In [3]:
#!pip install pyLDAvis 
"""
Ещё одним вариантом визуализации результатов работы алгоритма LDA в Python является модуль pyLDAvis, который позволяет
сохранять полученную тематическую модель в виде отдельного html файла для дальнейшей интерактивной работы с ней.
"""

  Using cached pyLDAvis-3.4.1-py3-none-any.whl (2.6 MB)
  Using cached joblib-1.3.1-py3-none-any.whl (301 kB)
  Using cached pandas-2.0.3-cp39-cp39-win_amd64.whl (10.8 MB)
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.4
    Uninstalling pandas-1.4.4:
      Successfully uninstalled pandas-1.4.4


# ДЗ №2

1) Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)

2) Повторить п.2, но используя уже не медиану, а max 

3) (опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.

4) Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score

5) Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

In [19]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\u187s\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [24]:
import pandas as pd
#from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

#предобработка текстов
import re
import numpy as np
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel
import pymorphy2  

from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix
from gensim.models import LdaModel
from gensim.test.utils import datapath
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [25]:
# прочитаем файл с перечнем статей, поданный в формате id/содержание
news = pd.read_csv("C:/Users/u187s/LEARNING/MLB/HWK_2/articles.csv")
print(news.shape)
news.head(5)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...
3,4898,Главный тренер «Кубани» Юрий Красножан прокомм...
4,4899,Решением попечительского совета владивостокско...


In [26]:
#Загрузим пользователей и списки последних прочитанных статей по doc_id
users = pd.read_csv('C:/Users/u187s/LEARNING/MLB/HWK_2/users_articles.csv')
users.head(5)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"
3,u101138,"[5933, 6186, 5055, 6977, 5206, 488389]"
4,u108248,"[707, 1144, 2532, 2928, 3133, 324592]"


In [27]:
# Необходимо получить векторные представления пользователей по прочитанным ими новостям и самих новостям:

stopword_ru = stopwords.words('russian')
print(len(stopword_ru))

morph = pymorphy2.MorphAnalyzer()

151


In [28]:
with open('C:/Users/u187s/LEARNING/MLB/HWK_2/stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [31]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str): #isinstance функция для проверки принодлежности обьекта конкретному типу.
        text = str(text)
    
    text = text.lower() 
    #меняем гегистр всех знаков
    text = text.strip('\n').strip('\r').strip('\t') 
    # удаление символов новой строки ('\n'), возврата каретки ('\r') и табуляции ('\t') с начала и конца строки текста.
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))
    """
    Данная строка кода использует модуль re (регулярные выражения) для замены определенных шаблонов подстрок в тексте
    на пустую строку.
    Используя функцию `re.sub()`, мы передаем три аргумента: регулярное выражение,
    заменитель и исходную строку.

    Регулярное выражение `-\s\r\n\|-\s\r\n|\r\n` содержит три шаблона, разделенных вертикальной чертой (`|`).    
    Каждый найденный шаблон заменяется на пустую строку (`''`) в коде.

    Данная строка кода заменяет все вхождения указанных шаблонов в исходном тексте на пустую строку.
    Это позволяет удалить эти шаблоны из текста.
    
    """
    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    """
    также удвление приведенных шаблонов из текста и их замена на строку
    """
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    """
    данная строка кода заменяет все вхождения шаблонов символов,
    связанных с разделителями строк и пробелами, на пробел. Это позволяет привести текст к единому формату,
    где эти символы заменены на пробелы
    """
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    """
     данная строка кода заменяет мягкий дефис и пробельные символы на пробелы,
     а затем удаляет пробельные символы из начала и конца текста.
     
     по итогу чистка текста это последовательность операций:
     1. все переводим в "текст"
     2. нижний регистр
     3. разбиение строки на слова, разделение их пустой строкой- как бы вертикальный список
     4. очистка от символов и их замена на строку
     5. удаление переносов и пробелов с помощью text.strip()
    """
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [32]:
%%time
#Запускаем очистку текста. 
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

Wall time: 17.1 s


In [33]:
%%time
#Запускаем лемматизацию текста. (приведение к словарной форме)
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

Wall time: 2min 12s


In [34]:
# Обучение модели:

#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [36]:
"""
Что такое common_dictionary и как он выглядит

common_dictionary[20]

таким образом можно вызвать любое слово из словаря
"""

'\nЧто такое common_dictionary и как он выглядит\n\ncommon_dictionary[20]\n\nтаким образом можно вызвать любое слово из словаря\n'

In [37]:
# Запускаем обучение:
#%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

In [38]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [39]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(3, 0.030961791),
 (8, 0.18403733),
 (9, 0.05148564),
 (10, 0.13074577),
 (18, 0.43100443),
 (24, 0.15486085)]

In [40]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: фонд млрд область рейтинг миллиард рубль губернатор
topic_1: километр берег оперативно мексиканский повторяться покой telegraphn
topic_2: женщина мужчина остров египет японский разместить вуз
topic_3: газ рынок снижение журнал сша товар американский
topic_4: поверхность северный высота фестиваль китай южный бомба
topic_5: тело смерть мужчина произойти женщина пилот инцидент
topic_6: год это который россия nn научный российский
topic_7: ii юрист литва оплачивать арабский миля калининградский
topic_8: год это новый который сша первый исследование
topic_9: год ребёнок который nn это исследование семья
topic_10: египетский лестница кит покупатель мвд натуральный радикально
topic_11: земля температура день болезнь тыс экипаж изз
topic_12: это который свой nn россия страна год
topic_13: год военный который это человек риск жизнь
topic_14: ракета компания станция продукция министерство суд выделить
topic_15: это который год млрд банк мочь российский
topic_16: налог двигатель форум ис

In [41]:
#пишем функцию, которая будет возвращать векторное представление новости:
#text = news['title'].iloc[0]

def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [42]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
1,4896,0.000000,0.0,0.031944,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.061631,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
2,4897,0.000000,0.0,0.000000,0.031609,0.0,0.0,0.0,0.0,0.182798,...,0.000000,0.0,0.000000,0.431157,0.000000,0.0,0.0,0.0,0.0,0.155176
3,4898,0.019246,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.308741,...,0.000000,0.0,0.000000,0.507308,0.128848,0.0,0.0,0.0,0.0,0.000000
4,4899,0.123839,0.0,0.327125,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.029735,0.062992,0.000000,0.0,0.0,0.0,0.0,0.000000


In [43]:
#векторные представления пользователей:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [44]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [45]:
doc_dict[293622]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.14263247, 0.        , 0.        , 0.26836744,
       0.        , 0.        , 0.        , 0.08507032, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.31316391, 0.01583997, 0.        , 0.16339324])

# ДЗ выполнение:

In [65]:
user_articles_list = users['articles'].iloc[33]

def get_user_embedding_mean(user_articles_list):
    user_articles_list = eval(user_articles_list)
    #print(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.mean(user_vector, 0)
    return user_vector

def get_user_embedding_median(user_articles_list):
    user_articles_list = eval(user_articles_list)
    #print(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.median(user_vector, 0)
    return user_vector

def get_user_embedding_max(user_articles_list):
    user_articles_list = eval(user_articles_list)
    #print(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = np.max(user_vector, 0)
    return user_vector

In [66]:
#получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче:
user_embeddings_mean = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding_mean(x), 1)])
user_embeddings_mean.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings_mean['uid'] = users['uid'].values
user_embeddings_mean = user_embeddings_mean[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings_mean.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.008607,0.000000,0.014783,0.000000,0.003802,0.007187,0.142565,0.0,0.116962,...,0.065907,0.000000,0.0,0.000000,0.051043,0.0,0.078017,0.00264,0.018218,0.128400
1,u108690,0.005086,0.001691,0.004073,0.005377,0.000000,0.010307,0.208288,0.0,0.010382,...,0.115645,0.000000,0.0,0.070446,0.017517,0.0,0.069127,0.00000,0.011893,0.098786
2,u108339,0.019085,0.000000,0.000000,0.009709,0.002451,0.035059,0.211021,0.0,0.025751,...,0.098630,0.004865,0.0,0.008983,0.126237,0.0,0.099202,0.00000,0.002825,0.011668


In [67]:
user_embeddings_median = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding_median(x), 1)])
user_embeddings_median.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings_median['uid'] = users['uid'].values
user_embeddings_median = user_embeddings_median[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings_median.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.000000,0.0,0.0,0.0,0.0,0.000000,0.100012,0.0,0.077730,...,0.000000,0.0,0.0,0.0000,0.000000,0.0,0.034745,0.0,0.000000,0.000000
1,u108690,0.000000,0.0,0.0,0.0,0.0,0.000000,0.211435,0.0,0.000000,...,0.091038,0.0,0.0,0.0626,0.008881,0.0,0.042217,0.0,0.005985,0.049875
2,u108339,0.016239,0.0,0.0,0.0,0.0,0.021426,0.201260,0.0,0.018033,...,0.077949,0.0,0.0,0.0000,0.110034,0.0,0.091271,0.0,0.000000,0.000000


In [68]:
user_embeddings_max = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding_max(x), 1)])
user_embeddings_max.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings_max['uid'] = users['uid'].values
user_embeddings_max = user_embeddings_max[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings_max.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.051641,0.000000,0.088700,0.000000,0.022810,0.043121,0.394285,0.0,0.314513,...,0.220558,0.000000,0.0,0.000000,0.220939,0.0,0.313164,0.01584,0.109310,0.607008
1,u108690,0.017823,0.010147,0.024435,0.032264,0.000000,0.032576,0.336639,0.0,0.046962,...,0.298279,0.000000,0.0,0.196915,0.052146,0.0,0.170825,0.00000,0.033259,0.294637
2,u108339,0.044921,0.000000,0.000000,0.035824,0.014708,0.111244,0.372324,0.0,0.092201,...,0.206815,0.029187,0.0,0.053900,0.304479,0.0,0.165631,0.00000,0.016948,0.047303


In [69]:
# попробуем обучить модель. Загрузим нашу разметку:
target = pd.read_csv('C:/Users/u187s/LEARNING/MLB/HWK_2/users_churn.csv')
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [70]:
X_mean = pd.merge(user_embeddings_mean, target, 'left')

X_median = pd.merge(user_embeddings_median, target, 'left')

X_max = pd.merge(user_embeddings_max, target, 'left')

In [71]:
#разделим данные на train/test
X_mean_train, X_mean_test, y_mean_train, y_mean_test = train_test_split(X_mean[['topic_{}'.format(i) for i in range(25)]], 
                                                    X_mean['churn'], random_state=0)

In [72]:
X_median_train, X_median_test, y_median_train, y_median_test = train_test_split(X_median[['topic_{}'.format(i) for i in range(25)]], 
                                                    X_median['churn'], random_state=0)

In [73]:
X_max_train, X_max_test, y_max_train, y_max_test = train_test_split(X_max[['topic_{}'.format(i) for i in range(25)]], 
                                                    X_max['churn'], random_state=0)

In [74]:
logreg_mean = LogisticRegression()
#обучим 
logreg_mean.fit(X_mean_train, y_mean_train)

LogisticRegression()

In [75]:
logreg_median = LogisticRegression()
#обучим 
logreg_median.fit(X_median_train, y_median_train)

LogisticRegression()

In [76]:
logreg_max = LogisticRegression()
#обучим 
logreg_max.fit(X_max_train, y_max_train)

LogisticRegression()

In [77]:
#наши прогнозы для тестовой выборки
preds_mean = logreg_mean.predict_proba(X_mean_test)[:, 1]
preds_mean[:10]

array([0.11638453, 0.03420878, 0.22226366, 0.26147397, 0.02891024,
       0.09234556, 0.24190505, 0.06324814, 0.22810525, 0.24579717])

In [78]:
#наши прогнозы для тестовой выборки
preds_median = logreg_median.predict_proba(X_median_test)[:, 1]
preds_median[:10]

array([0.1854918 , 0.03911334, 0.41237093, 0.20042129, 0.0467526 ,
       0.11214018, 0.22934422, 0.04852276, 0.08045623, 0.2050416 ])

In [79]:
#наши прогнозы для тестовой выборки
preds_max = logreg_max.predict_proba(X_max_test)[:, 1]
preds_max[:10]

array([0.02007045, 0.00365139, 0.69759375, 0.32718245, 0.00812731,
       0.03603509, 0.08013866, 0.04093922, 0.02715155, 0.19429831])

In [80]:
model = ['mean', 'median', 'max']
precision = []
recall = []
fscore = []
roc_auc = []

In [81]:
#Рассчитаем Precision, Recall, F_score:
precision_mean, recall_mean, thresholds_mean = precision_recall_curve(y_mean_test, preds_mean)
fscore_mean = (2 * precision_mean * recall_mean) / (precision_mean + recall_mean)
# locate the index of the largest f score
ix_mean = np.argmax(fscore_mean)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds_mean[ix_mean], 
                                                                        fscore_mean[ix_mean],
                                                                        precision_mean[ix_mean],
                                                                        recall_mean[ix_mean]))
precision.append(precision_mean[ix_mean])
recall.append(recall_mean[ix_mean])
fscore.append(fscore_mean[ix_mean])

Best Threshold=0.225925, F-Score=0.587, Precision=0.506, Recall=0.698


In [82]:
#Рассчитаем Precision, Recall, F_score:
precision_median, recall_median, thresholds_median = precision_recall_curve(y_median_test, preds_median)
fscore_median = (2 * precision_median * recall_median) / (precision_median + recall_median)
# locate the index of the largest f score
ix_median = np.argmax(fscore_median)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds_median[ix_median], 
                                                                        fscore_median[ix_median],
                                                                        precision_median[ix_median],
                                                                        recall_median[ix_median]))
precision.append(precision_median[ix_median])
recall.append(recall_median[ix_median])
fscore.append(fscore_median[ix_median])

Best Threshold=0.771111, F-Score=nan, Precision=0.000, Recall=0.000


C:\Users\u187s\AppData\Local\Temp\ipykernel_18532\3521988928.py:3: RuntimeWarning: invalid value encountered in divide
  fscore_median = (2 * precision_median * recall_median) / (precision_median + recall_median)


In [83]:
#Рассчитаем Precision, Recall, F_score:
precision_max, recall_max, thresholds_max = precision_recall_curve(y_max_test, preds_max)
fscore_max = (2 * precision_max * recall_max) / (precision_max + recall_max)
# locate the index of the largest f score
ix_max = np.argmax(fscore_max)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds_max[ix_max], 
                                                                        fscore_max[ix_max],
                                                                        precision_max[ix_max],
                                                                        recall_max[ix_max]))
precision.append(precision_max[ix_max])
recall.append(recall_max[ix_max])
fscore.append(fscore_max[ix_max])

Best Threshold=0.314471, F-Score=0.751, Precision=0.722, Recall=0.784


In [84]:
roc_auc_mean = roc_auc_score(y_mean_test, preds_mean)
print(f'mean: {roc_auc_mean}')
roc_auc.append(roc_auc_mean)

roc_auc_median = roc_auc_score(y_median_test, preds_median)
print(f'median: {roc_auc_median}')
roc_auc.append(roc_auc_median)

roc_auc_max = roc_auc_score(y_max_test, preds_max)
print(f'max: {roc_auc_max}')
roc_auc.append(roc_auc_max)

mean: 0.8995895110180826
median: 0.9472039072039072
max: 0.9646839932554218


In [85]:
results = pd.DataFrame(np.column_stack([model, precision, recall, fscore, roc_auc]), 
                               columns=['model', 'precision', 'recall', 'fscore', 'roc_auc'])

In [86]:
results

,model,precision,recall,fscore,roc_auc
0,mean,0.5059171597633136,0.6979591836734694,0.5866209262435678,0.8995895110180826
1,median,0.0,0.0,nan,0.9472039072039072
2,max,0.7218045112781954,0.7836734693877551,0.7514677103718199,0.9646839932554218


Самый хороший результат показала модель, в которую были поданы максимальные значения векторов. Возможно это связано с тем, что мы берем максимально характеризующие пользователя вектора статей (тем самым "сужая допуски" модели при обучении) из чего она делает более точные прогнозы так как базово обучалась на наиболее подходящих значениях.

In [89]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [90]:
#мы уже нашли ранее "оптимальный" порог, когда максимизировали f_score
font = {'size' : 15}

plt.rc('font', **font)

cnf_matrix = confusion_matrix(y_test, preds>thresholds[ix])
plt.figure(figsize=(10, 8))
plot_confusion_matrix(cnf_matrix, classes=['Non-Churn', 'churn'],
                      title='Confusion matrix')
plt.savefig("conf_matrix.png")
plt.show()

NameError: name 'y_test' is not defined